In [24]:
import pyomo.environ as pyo
import pandas as pd

In [25]:
df = pd.read_csv('problem_3_network_dataset.csv')
df.head()

,node_1,node_2,distance
0,1,2,20
1,1,4,8
2,2,3,15
3,2,6,22
4,2,7,7


In [26]:
# node에 대한 값을 unique하게 정의해서 set으로 구성
nodes = list(set(df['node_1']).union(set(df['node_2'])))
# arc에 대한 값을 tuple로 구성
arcs = list(zip(df['node_1'], df['node_2']))

# 거리에 대한 값을 dictionary로 정의
dist_dict = {(row.node_1, row.node_2): row.distance for idx, row in df.iterrows()}
# 양방향 계산을 위해서 역방향에 대한 vector로 추가
for idx, row in df.iterrows():
    dist_dict[(row.node_2, row.node_1)] = row.distance
    arcs.append((row.node_2, row.node_1))
arcs = list(set(arcs))

In [27]:
# model 정의
model = pyo.ConcreteModel()

# 변수 집합
model.Nodes = pyo.Set(initialize=nodes)
model.Arcs = pyo.Set(initialize=arcs, dimen=2)

# 변수 정의
model.x = pyo.Var(model.Arcs, domain=pyo.Binary)

# 목적함수
def obj_rule(m):
    return sum(dist_dict[i,j]*m.x[i,j] for (i,j) in m.Arcs)
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# 흐름 제약식
def flow_rule(m, n):
    if n == 1:
        return sum(m.x[i, j] for (i, j) in m.Arcs if i == n) - sum(m.x[j, i] for (j, i) in m.Arcs if i == n) == 1
    elif n == 30:
        return sum(m.x[i, j] for (i, j) in m.Arcs if j == n) - sum(m.x[j, i] for (j, i) in m.Arcs if j == n) == 1
    else:
        return sum(m.x[i, j] for (i, j) in m.Arcs if i == n) - sum(m.x[j, i] for (j, i) in m.Arcs if i == n) == 0
model.flow = pyo.Constraint(model.Nodes, rule=flow_rule)

In [28]:
# MILP 풀기
solver = pyo.SolverFactory('ipopt')
result = solver.solve(model, tee=False)

In [29]:
# 결과 해석
if (result.solver.status == pyo.SolverStatus.ok) and (result.solver.termination_condition == pyo.TerminationCondition.optimal):
    print("최적해를 찾았습니다.")
    # 경로 추적
    path = [1]
    current = 1
    while current != 30:
        for (i, j) in model.Arcs:
            if i == current and pyo.value(model.x[i, j]) > 0.5:
                path.append(j)
                current = j
                break
    print("최단 경로:", path)
    print("최소 거리:", pyo.value(model.obj))
else:
    print("최적해를 찾지 못했습니다.")

최적해를 찾았습니다.
최단 경로: [1, 2, 3, 16, 26, 27, 30]
최소 거리: 142.9999731747019
